In [ ]:
# Notebook A: Classification avec Machine Learning classique
# Classification des chiffres manuscrits avec Random Forest

# Partie 1: Importation des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import time
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.pipeline import Pipeline
from google.colab import output
output.enable_custom_widget_manager()
import ipywidgets as widgets

# Partie 2: Chargement et exploration des données
print("Chargement du jeu de données MNIST...")
# Utilisation du jeu de données MNIST intégré à sklearn
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"]
X = X / 255.0  # Normalisation des valeurs de pixels entre 0 et 1
y = y.astype(np.uint8)  # Conversion des labels en entiers

# Exploration des données
print(f"Dimensions du jeu de données: {X.shape}")
print(f"Nombre de classes: {len(np.unique(y))}")

# Affichage de quelques exemples
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X[i].reshape(28, 28), cmap='gray')
    plt.title(f"Label: {y[i]}")
    plt.axis('off')
plt.tight_layout()
plt.suptitle("Exemples de chiffres manuscrits", y=1.05)
plt.show()

# Partie 3: Préparation des données pour Machine Learning classique

print("\n--- Préparation des données pour Random Forest ---")
print("Pour le Machine Learning classique, nous devons souvent extraire des caractéristiques manuellement.")

# Réduction de dimension avec PCA pour accélérer l'entraînement
print("Application d'une réduction de dimension (PCA)...")
n_components = 50  # Réduire de 784 à 50 caractéristiques

# Séparation en ensembles d'entraînement et de test
# Utilisation d'un échantillon réduit pour accélérer la démonstration
X_sample = X[:10000]
y_sample = y[:10000]

X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

print(f"Taille de l'ensemble d'entraînement: {X_train.shape}")
print(f"Taille de l'ensemble de test: {X_test.shape}")

# Création d'un pipeline d'extraction de caractéristiques
feature_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=n_components))
])

# Application aux données
print("Extraction de caractéristiques...")
X_train_features = feature_pipeline.fit_transform(X_train)
X_test_features = feature_pipeline.transform(X_test)

print(f"Dimensions après extraction de caractéristiques: {X_train_features.shape}")

# Partie 4: Entraînement du modèle Random Forest

print("\n--- Entraînement du modèle Random Forest ---")

# Paramètres du modèle - vous pouvez les modifier
n_estimators = 100  # Nombre d'arbres
max_depth = 10      # Profondeur maximale des arbres
min_samples_split = 2  # Nombre minimum d'échantillons requis pour diviser un nœud

# Création du modèle
rf_model = RandomForestClassifier(
    n_estimators=n_estimators,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    random_state=42,
    n_jobs=-1  # Utiliser tous les cœurs disponibles
)

# Mesure du temps d'entraînement
start_time = time.time()
print("Entraînement du modèle en cours...")
rf_model.fit(X_train_features, y_train)
end_time = time.time()
training_time = end_time - start_time

print(f"Temps d'entraînement: {training_time:.2f} secondes")

# Partie 5: Évaluation du modèle

print("\n--- Évaluation du modèle Random Forest ---")

# Prédictions sur l'ensemble de test
y_pred = rf_model.predict(X_test_features)

# Calcul des métriques
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Précision globale: {accuracy*100:.2f}%")
print("\nMatrice de confusion:")
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Valeurs réelles')
plt.title('Matrice de confusion')
plt.show()

print("\nRapport de classification:")
print(class_report)

# Partie 6: Visualisation des erreurs

print("\n--- Analyse des erreurs ---")

# Identifier les erreurs
error_indices = np.where(y_pred != y_test)[0]
n_errors = min(10, len(error_indices))  # Afficher max 10 erreurs

if n_errors > 0:
    plt.figure(figsize=(12, 4))
    for i, idx in enumerate(error_indices[:n_errors]):
        plt.subplot(2, 5, i + 1)
        # Récupérer l'image originale
        img = X_test[idx].reshape(28, 28)
        plt.imshow(img, cmap='gray')
        plt.title(f"Réel: {y_test[idx]}\nPrédit: {y_pred[idx]}")
        plt.axis('off')
    plt.tight_layout()
    plt.suptitle("Exemples d'erreurs de classification", y=1.05)
    plt.show()
else:
    print("Aucune erreur trouvée dans l'échantillon de test!")

# Partie 7: Importance des caractéristiques

print("\n--- Importance des caractéristiques ---")
# Visualiser l'importance des composantes principales
feature_importance = rf_model.feature_importances_
sorted_idx = np.argsort(feature_importance)[::-1]

plt.figure(figsize=(10, 5))
plt.bar(range(20), feature_importance[sorted_idx[:20]])
plt.xticks(range(20), [f"Feat {i}" for i in sorted_idx[:20]], rotation=90)
plt.xlabel('Composantes principales')
plt.ylabel('Importance')
plt.title('Top 20 des composantes principales les plus importantes')
plt.tight_layout()
plt.show()

print("Les caractéristiques les plus importantes sont les composantes principales qui capturent le plus de variance dans les données.")

# Partie 8: Défi de généralisation

print("\n--- Défi de généralisation ---")
print("Nous allons maintenant tester le modèle sur des chiffres légèrement modifiés pour évaluer sa capacité de généralisation.")

# Fonction pour ajouter du bruit aux images
def add_noise(images, noise_level=0.2):
    noisy_images = images.copy()
    noise = np.random.normal(0, noise_level, images.shape)
    noisy_images = noisy_images + noise
    # Assurer que les valeurs restent entre 0 et 1
    noisy_images = np.clip(noisy_images, 0, 1)
    return noisy_images

# Fonction pour appliquer une rotation aux images
def rotate_images(images, max_angle=15):
    from scipy.ndimage import rotate
    rotated_images = np.zeros_like(images)
    for i, img in enumerate(images):
        angle = np.random.uniform(-max_angle, max_angle)
        img_2d = img.reshape(28, 28)
        rotated = rotate(img_2d, angle, reshape=False)
        rotated_images[i] = rotated.flatten()
    return rotated_images

# Créer un jeu de données modifié
print("Création d'un jeu de données avec des chiffres modifiés...")

# Utiliser la partie restante des données pour ce test
X_new = X[10000:12000]
y_new = y[10000:12000]

# Appliquer des transformations
X_new_noisy = add_noise(X_new, noise_level=0.2)
X_new_rotated = rotate_images(X_new, max_angle=15)

# Visualiser quelques exemples
plt.figure(figsize=(12, 8))
for i in range(5):
    # Original
    plt.subplot(3, 5, i + 1)
    plt.imshow(X_new[i].reshape(28, 28), cmap='gray')
    plt.title(f"Original: {y_new[i]}")
    plt.axis('off')
    
    # Avec bruit
    plt.subplot(3, 5, i + 6)
    plt.imshow(X_new_noisy[i].reshape(28, 28), cmap='gray')
    plt.title("Avec bruit")
    plt.axis('off')
    
    # Avec rotation
    plt.subplot(3, 5, i + 11)
    plt.imshow(X_new_rotated[i].reshape(28, 28), cmap='gray')
    plt.title("Avec rotation")
    plt.axis('off')

plt.tight_layout()
plt.suptitle("Exemples de chiffres modifiés", y=1.02)
plt.show()

# Évaluer le modèle sur les données modifiées
print("\nÉvaluation sur les données avec bruit:")
X_new_noisy_features = feature_pipeline.transform(X_new_noisy)
y_new_noisy_pred = rf_model.predict(X_new_noisy_features)
accuracy_noisy = accuracy_score(y_new, y_new_noisy_pred)
print(f"Précision sur données bruitées: {accuracy_noisy*100:.2f}%")

print("\nÉvaluation sur les données avec rotation:")
X_new_rotated_features = feature_pipeline.transform(X_new_rotated)
y_new_rotated_pred = rf_model.predict(X_new_rotated_features)
accuracy_rotated = accuracy_score(y_new, y_new_rotated_pred)
print(f"Précision sur données pivotées: {accuracy_rotated*100:.2f}%")

print("\nComparaison avec la précision originale:")
print(f"Précision sur données originales: {accuracy*100:.2f}%")
print(f"Précision sur données bruitées: {accuracy_noisy*100:.2f}%")
print(f"Précision sur données pivotées: {accuracy_rotated*100:.2f}%")

# Partie 9: Conclusions et réflexion

print("\n--- Conclusions sur le Machine Learning classique ---")
print("""
Points forts du Random Forest:
- Entraînement relativement rapide
- Bonnes performances sur les données originales
- Interprétabilité (importance des caractéristiques)

Limites:
- Nécessite une extraction manuelle de caractéristiques (PCA dans notre cas)
- Sensibilité aux transformations des données (bruit, rotation)
- Difficulté à capturer des motifs complexes sans feature engineering approprié

Questions pour la réflexion:
1. Pourquoi avons-nous besoin de réduire la dimensionnalité pour le Random Forest?
2. Comment pourrait-on améliorer la robustesse aux transformations?
3. Quelles autres caractéristiques pourraient être extraites manuellement pour améliorer les performances?
""")

# Partie 10: Widget interactif pour tester le modèle

print("\n--- Testez le modèle vous-même ---")

def test_model(digit_idx):
    if digit_idx < len(X_test):
        # Afficher l'image
        img = X_test[digit_idx].reshape(28, 28)
        plt.figure(figsize=(6, 6))
        plt.imshow(img, cmap='gray')
        plt.title(f"Chiffre à classifier")
        plt.axis('off')
        plt.show()
        
        # Faire la prédiction
        features = feature_pipeline.transform([X_test[digit_idx]])
        prediction = rf_model.predict(features)[0]
        real_label = y_test[digit_idx]
        
        print(f"Prédiction du modèle Random Forest: {prediction}")
        print(f"Étiquette réelle: {real_label}")
        print(f"Prédiction {'correcte' if prediction == real_label else 'incorrecte'}")
    else:
        print("Index hors limites!")

# Créer un slider pour sélectionner un chiffre à tester
digit_selector = widgets.IntSlider(
    value=0,
    min=0,
    max=len(X_test)-1,
    step=1,
    description='Index:',
    continuous_update=False
)

# Bouton pour exécuter le test
test_button = widgets.Button(description="Tester")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        test_model(digit_selector.value)

test_button.on_click(on_button_clicked)

# Afficher les widgets
display(widgets.HBox([digit_selector, test_button]))
display(output)

print("Utilisez le slider pour sélectionner un index et cliquez sur 'Tester' pour classifier le chiffre correspondant.")